In [1]:
from uproot_io import Events, View
import numpy as np
from numpy import *
import matplotlib
import matplotlib.pyplot as plt
import math
titlesize = 20
labelsize = 14

In [2]:
events = Events(r"C:\Users\andre\Desktop\Physics Project\bigdata\CheatedRecoFile_1.root")

In [7]:
testing = np.load("testing_events.npy")
ratioclosetrk = np.load("ratioclose_trk.npy")
ratiocloseshw = np.load("ratioclose_shw.npy")
connecttrk = np.load("connect_trk.npy")
connectshw = np.load("connect_shw.npy")
energytrk = np.load("energy_trk.npy")
energyshw = np.load("energy_shw.npy")
variencetrk = np.load("varience_trk.npy")
varienceshw = np.load("varience_shw.npy")
symmetrytrk = np.load("symmetry_trk.npy")
symmetryshw = np.load("symmetry_shw.npy")

In [15]:
def fit_line(x, y):
    A = np.vstack([x, np.ones_like(x)]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def calculate_residuals(x, y, m, c):
    #y_predicted = (m*x) + c
    perpdist = abs(((-m*x) + y - c) / np.sqrt(m**2 + 1))
    #residuals = np.abs(perpdist - y_predicted)
    return perpdist

def coords(event_number):
    idx = np.where(events.event_number == event_number)[0]
    gradients = []
    intercepts = []
    particleType = []
    ratio = []
    results = []
    allSym = []
    
    xmax = -1000
    xmin = 1000
    zmax = -1000
    zmin = 1000
    for i in idx:
        x0 = events.reco_hits_x_w[i]
        z0 = events.reco_hits_w[i]
        if len(x0) > 0:
            if max(x0) >= xmax:
                xmax = max(x0)
            if min(x0) <= xmin:
                xmin = min(x0)
        if len(z0) > 0:
            if max(z0) >= zmax:
                zmax = max(z0)
            if min(z0) <= zmin:
                zmin = min(z0)

    xlen = 0.1*(xmax - xmin)
    zlen = 0.1*(zmax - zmin)
    
    for i in idx:
        x = events.reco_hits_x_w[i]
        z = events.reco_hits_w[i]
        pdg = events.mc_pdg[i]
        purity = events.purity[i]
        complete = events.completeness[i]
        
        if len(x) > 10 and pdg != 0 and purity > 0.75 and complete > 0.75:
            #particleType.append(pdg)
            
            best_inliers = []
            best_model = None
            sample_size = 10
            max_iterations = 100

            for _ in range(max_iterations):
                sample_size = min(sample_size, len(x))
                sample_indices = np.random.choice(len(x), size=sample_size, replace=False)
                x_sample = x[sample_indices]
                z_sample = z[sample_indices]

                m, c = fit_line(x_sample, z_sample)
                residuals = calculate_residuals(x, z, m, c)
                inliers = np.where(residuals < (0.15*xlen))[0]

                if len(inliers) > len(best_inliers):
                    best_inliers = inliers
                    best_model = (m, c)

            if len(best_inliers) > 0:
                m, c = best_model
                gradients.append(m)
                intercepts.append(c)
                particleType.append(pdg)
                
                znew = z - c
                angle = -math.atan(m)
                l = x * np.cos(angle) - znew * np.sin(angle)
                t = x * np.sin(angle) + znew * np.cos(angle)

                
                tdiff = np.abs(max(t)-min(t))
                margin = tdiff*0.05
                inside = [x for x in t if x <= margin and x >= -margin]
                ratio.append(len(inside)/len(t))
                
                
                p1 = int(0.7 * len(t))
                X = t[p1:]
                Xvar = np.var(X)
                
                p2 = int(0.3 * len(t))
                Y = t[:p2]
                Yvar = np.var(Y)
                
                if  Yvar > Xvar:
                    resultY = np.sqrt(Yvar)
                    results.append(resultY)           
                else:
                    resultX = np.sqrt(Xvar)
                    results.append(resultX)
                
                
                above = np.sum(t > 0)
                below = np.sum(t < 0)
                
                lmin, lmax = np.min(l), np.max(l)
                length = lmax - lmin

                if above == 0 or below == 0 or length == 0:
                    allSym.append(2)
                    
                else:
                    # If symmetric, this should be close to 1, otherwise > 1
                    sym = (above + below) / length
                    allSym.append(sym)
            
            else:
                ratio.append(0.1)
                results.append(1.5)
                allSym.append(2)
                particleType.append(pdg)
    
    return particleType, ratio, results, allSym

In [16]:
def connectiondist(event_number):
    totals = []
    parts = []
    
    idx = np.where(events.event_number == event_number)[0]
    newidx = idx - idx[0]
    for i in newidx:
        x = events.reco_hits_x_w[i+idx[0]]
        z = events.reco_hits_w[i+idx[0]]
        part = events.mc_pdg[i+idx[0]]
        purity = events.purity[i+idx[0]]
        complete = events.completeness[i+idx[0]]
        
        if len(x) > 10 and part != 0 and purity > 0.75 and complete > 0.75 and (len(x)-1) != 0:
            total_distance = 0.0
            for j in range(len(x) - 1):
                distance = np.sqrt((x[j+1] - x[j])**2 + (z[j+1] - z[j])**2)
                total_distance += distance
            meandist = total_distance/(len(x)-1)
            totals.append(meandist)
            parts.append(part)
    
    return np.array(totals), np.array(parts)

In [17]:
def energy(event_number):    
    particleType = np.array([])
    E = np.array([])
    
    idx = np.where(events.event_number == event_number)[0]
    for i in idx:
        x = events.reco_hits_x_w[i]
        z = events.reco_hits_w[i]
        adc = events.reco_adcs_w[i]        
        purity = events.purity[i]
        complete = events.completeness[i]
        pdg = events.mc_pdg[i]
        
        if len(x) > 10 and pdg != 0 and purity > 0.75 and complete > 0.75:
            percent = int(0.8 * len(adc))
            X = adc[percent:]
            Xsum = np.sum(X)
            
            total = np.sum(adc)
            result = Xsum / total
            
            if result != 1.0:
                E = np.append(E, result)
                particleType = np.append(particleType, pdg) 
            
    return E, particleType

In [26]:
def likelihood():
    trackprob = []
    showerprob = []
    
    ratiobin = np.arange(0, 1.05, 0.05)
    totalbin = np.arange(0, 15.5, 0.5)
    volbin = np.arange(0, 1.05, 0.05)
    resultsbin = np.arange(0, 10.5, 0.5)
    allSymbin = np.arange(0, 10.5, 0.5)
    
    for n in testing:
        part, ratio, results, allSym = coords(n)
        total, _ = connectiondist(n)
        vol, _ = energy(n)
        
        bin1 = np.searchsorted(ratiobin, ratio, side='left') - 1
        bin2 = np.searchsorted(totalbin, total, side='left') - 1
        bin3 = np.searchsorted(volbin, vol, side='left') - 1
        bin4 = np.searchsorted(resultsbin, results, side='left') - 1
        bin5 = np.searchsorted(allSymbin, allSym, side='left') - 1
        
        print(bin1)
        
        trk1 = ratioclosetrk[bin1]
        trk2 = connecttrk[bin2]
        trk3 = energytrk[bin3]
        trk4 = variencetrk[bin4]
        trk5 = symmetrytrk[bin5]
        trackprob.append(trk1*trk2*trk3*trk4*trk5)
        
        shw1 = ratiocloseshw[bin1]
        shw2 = connectshw[bin2]
        shw3 = energyshw[bin3]
        shw4 = varienceshw[bin4]
        shw5 = symmetryshw[bin5]
        showerprob.append(shw1*shw2*shw3*shw4*shw5)
    
    return part, trackprob, showerprob

In [27]:
likelihood()

[1 2 7 1 2]
[9 4]


IndexError: index 30 is out of bounds for axis 0 with size 30